In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import affine_transform, geometric_transform

from reconstruction import create_nifti_image
from registration.reg_utils import loadNiiImages
import registration as rg


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
from scipy import stats

In [3]:
templateImagePath = "CCF_DATA/average_template_25.nii.gz"
dataImagePath = "../registration/B39/brain_25.nii.gz"
annotationImagePath = "CCF_DATA/annotation_25.nii.gz"

In [4]:
fixedImagePath = templateImagePath
movingImagePath = dataImagePath

In [5]:
A, _ = rg.axisAlignData(fixedImagePath, movingImagePath)

100%|███████████████████████████████████████████████████████████████████████████████| 528/528 [00:02<00:00, 242.54it/s]


In [6]:
fdata , mdata = loadNiiImages([fixedImagePath,movingImagePath ])

In [4]:
regDir ="reg"

elastixX =  os.path.join(os.path.join(regDir, "elastixX"), "result.nii")
elastixY =  os.path.join(os.path.join(regDir, "elastixY"), "result.nii")
elastixZ =  os.path.join(os.path.join(regDir, "elastixZ"), "result.nii")


deformationField = np.load(os.path.join(regDir,"deformation3d.npy"))
finalX   = rg.applyDeformationField(elastixX , deformationField)
finalY   = rg.applyDeformationField(elastixY , deformationField)
finalZ   = rg.applyDeformationField(elastixZ , deformationField)

mapping = np.zeros((fx,fy,fz,3))
mapping[:,:,:, 0] = finalX

mapping[:,:,:, 1] = finalY

mapping[:,:,:, 2] = finalZ
mapping[mapping<0] = 0

KeyboardInterrupt: 

In [8]:
mapping  = np.random.rand(200,300,450,3)*100

In [9]:
np.argmin(mapping - np.reshape(([25,35,67]), (1,1,1,3)))

77518733

In [27]:
x

166

In [21]:
np.argmin(mnorm,axis=2)

array([[120,  21, 271, ..., 321,  82, 165],
       [274,  31,  93, ..., 218, 282, 185],
       [192,   4, 371, ...,   1, 182, 358],
       ...,
       [ 66, 280,   8, ..., 384,  91,   6],
       [220, 235, 233, ..., 166, 433, 246],
       [128, 427, 423, ...,  59, 220, 303]], dtype=int64)

In [14]:
np.min(mnorm)

0.24972070834454294

In [7]:
outputDir = "reg"

def loadTransformMapping(fshape, mshape,regDir):
    """
    Returns the mapping of shape fshapex3. 
    For each voxel located at x,y,z in fdata, the 1x3 cell represents the x_,y_,z_ in mdata. 
    """
    
    mx, my, mz = mshape
    fx, fy, fz = fshape
    
    #Sample volumes that represent the coordinate of the voxel
    X, Y,Z= np.meshgrid(range(mx), range(my), range(mz), indexing = 'ij')
    X =X.astype(float)
    Y =Y.astype(float)
    Z= Z.astype(float)
    
    alignedX = affine_transform(X,np.linalg.inv(A), output_shape = X.shape, order =1)
    alignedY = affine_transform(Y,np.linalg.inv(A), output_shape = Y.shape, order =1)
    alignedZ = affine_transform(Z,np.linalg.inv(A), output_shape = Z.shape, order =1)
    alignedXPath = os.path.join(regDir, "alignX.nii.gz")
    alignedYPath = os.path.join(regDir, "alignY.nii.gz")
    alignedZPath = os.path.join(regDir, "alignZ.nii.gz")
    
    create_nifti_image(alignedX, 2.5, alignedXPath, 1)
    create_nifti_image(alignedY, 2.5, alignedYPath, 1)
    create_nifti_image(alignedZ, 2.5, alignedZPath, 1)

    elastixX =  rg.elastixTransformation(alignedXPath, regDir, os.path.join(regDir, "elastixX"))
    elastixY =  rg.elastixTransformation(alignedYPath, regDir, os.path.join(regDir, "elastixY"))
    elastixZ =  rg.elastixTransformation(alignedZPath, regDir, os.path.join(regDir, "elastixZ"))

    deformationField = np.load(os.path.join(regDir,"deformation3d.npy"))
    finalX   = rg.applyDeformationField(elastixX , deformationField)
    finalY   = rg.applyDeformationField(elastixY , deformationField)
    finalZ   = rg.applyDeformationField(elastixZ , deformationField)
    
    mapping = np.zeros((fx,fy,fz,3))
    mapping[:,:,:, 0] = finalX

    mapping[:,:,:, 1] = finalY

    mapping[:,:,:, 2] = finalZ
    mapping[mapping<0] = 0
    
    #Remove all junk files
    os.remove(os.path.join(regDir, "alignX.nii.gz"))
    os.remove(os.path.join(regDir, "alignY.nii.gz"))
    os.remove(os.path.join(regDir, "alignZ.nii.gz"))
    shutil.rmtree(os.path.join(regDir, "elastixX"))
    shutil.rmtree(os.path.join(regDir, "elastixY"))
    shutil.rmtree(os.path.join(regDir, "elastixZ"))
    
    return mapping

In [104]:
anndata = loadNiiImages([annotationImagePath])

In [73]:
mx, my, mz = mdata.shape

In [ ]:
def getPointRegions(points, mapping):
    mint = mapping.astype(int)
    pointRegions =[]
    for point in points:
        x,y,z = point
        indices = np.where(np.bitwise_and(np.bitwise_and(mint[:,:,:,0]==x, mint[:,:,:,1]==y), mint[:,:,:,2]==z))
        if(len(indices[0]) ==0 ):
            pointRegions.append(0)
        else:
            pointRegions.append(stats.mode(anndata[indices[0], indices[1], indices[2]], keepdims=False).mode)
    return pointRegions

In [135]:
mint = mapping.astype(int)
fx, fy, fz  = fdata.shape
mapRev = np.zeros((mx,my, mz,3))
for x in tqdm(range(fx)):
    for y in range(fy):
        for z in range(fz):
            x_,y_, z_ = mint[x,y,z]
            mapRev[x_,y_,z_] = anndata[x,y,z]

100%|████████████████████████████████████████████████████████████████████████████████| 528/528 [04:25<00:00,  1.99it/s]


In [136]:
create_nifti_image(mapRev, 2.5, "map.nii.gz", 1)

In [134]:
for x in tqdm(range(mx)):
    for y in tqdm(range(my)):
        for z in range(mz):
            indices = np.where(np.bitwise_and(np.bitwise_and(mint[:,:,:,0]==x, mint[:,:,:,1]==y), mint[:,:,:,2]==z))
            if(len(indices[0]) ==0):
                continue
            print("hey" , indices)
            mapRev[x,y,z] = stats.mode(anndata[indices[0], indices[1], indices[2]], keepdims=False).mode

  0%|                                                                                          | 0/436 [00:00<?, ?it/s]

hey (array([  0,   0,   0, ..., 527, 527, 527], dtype=int64), array([ 60,  60,  60, ..., 319, 319, 319], dtype=int64), array([132, 133, 134, ..., 257, 258, 259], dtype=int64))
hey (array([ 0,  0,  0, ..., 30, 30, 30], dtype=int64), array([ 60,  60,  60, ..., 280, 280, 280], dtype=int64), array([111, 112, 113, ..., 264, 265, 266], dtype=int64))
hey (array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3

hey (array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 5, 5, 5, 7], dtype=int64), array([ 60,  60,  60,  60,  60,  60,  60,  61,  61,  61,  61,  62,  62,
        62,  62,  62,  62,  62,  62,  67,  67,  67,  67,  67,  67,  69,
        69,  69,  69,  69,  70,  74,  74,  74,  74,  76,  76,  76,  76,
        82,  83,  85,  85,  85,  85,  85,  85,  85,  85,  85,  85,  85,
        85,  86,  86,  86,  86,  86,  81,  81,  81,  83,  83,  83,  91,
        91,  91,  92,  94,  94, 174, 107, 109, 109, 122], dtype=int64), array([202, 203, 204, 205, 206, 207, 208, 238, 239, 240, 241, 199, 200,
       201, 202, 203, 204, 205, 206, 210, 211, 212, 213, 214, 242, 209,
       210, 211, 212, 213, 239, 215, 216, 217, 218, 217, 218, 219, 220,
       219, 234, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258,
       

  0%|                                                                                          | 0/280 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [120]:
indices = np.where(np.bitwise_and(np.bitwise_and(mint[:,:,:,0]==145, mint[:,:,:,1]==267), mint[:,:,:,2]==289))

In [122]:
mint.shape

(528, 320, 456, 3)

In [121]:
indices

(array([303, 304], dtype=int64),
 array([236, 236], dtype=int64),
 array([221, 221], dtype=int64))

In [106]:
indices[0]

(array([288, 288, 289], dtype=int64),
 array([191, 192, 192], dtype=int64),
 array([184, 184, 184], dtype=int64))

In [124]:
stats.mode(anndata[indices[0], indices[1], indices[2]], keepdims = False).mode

515.0

In [119]:
anndata[indices[0], indices[1], indices[2]]

array([0., 0., 0., ..., 0., 0., 0.])

In [110]:
anndata[288,191,184]

715.0

In [113]:
import SimpleITK as sitk

In [114]:
annotationImage  = sitk.ReadImage(annotationImagePath)

In [ ]:
annotationImage.GetPixel()

In [111]:
528-288

240

In [80]:
m = mapping - np.array([5,6,7])

In [99]:
np.where(mint[:,:,:,2]==7)

(array([ 0,  0,  0, ..., 37, 37, 37], dtype=int64),
 array([ 59,  59,  59, ..., 319, 319, 319], dtype=int64),
 array([112, 113, 114, ..., 168, 169, 170], dtype=int64))

In [95]:
np.min(abs(mint[:,:,:,0]- 5))

0

In [100]:
mdata.shape

(280, 436, 562)

In [102]:
indices = np.where(np.bitwise_and(np.bitwise_and(mint[:,:,:,0]==140, mint[:,:,:,1]==225), mint[:,:,:,2]==256))

3

In [85]:
np.where((abs(m[:,:,:,0])<0.5) &  (abs(m[:,:,:,1])<0.5) & (abs(m[:,:,:,2])<0.5))

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))

In [ ]:
for x in range(mx):
    for y in range(my):
        for z in range(mz):
            abs(mapping[:,:,:])

In [65]:
mapping[30, 45, 53]

array([ 16.31915092,  54.55852127, 138.71694946])

In [8]:
%%time
alignedX = affine_transform(X,np.linalg.inv(A), output_shape = X.shape, order =1)
alignedY = affine_transform(Y,np.linalg.inv(A), output_shape = Y.shape, order =1)
alignedZ = affine_transform(Z,np.linalg.inv(A), output_shape = Z.shape, order =1)

CPU times: total: 30.2 s
Wall time: 30.2 s


In [50]:
%%time
outputDir = "reg"
alignedXPath = os.path.join(outputDir, "alignX.nii.gz")
alignedYPath = os.path.join(outputDir, "alignY.nii.gz")
alignedZPath = os.path.join(outputDir, "alignZ.nii.gz")
create_nifti_image(alignedX, 2.5, alignedXPath, 1)
create_nifti_image(alignedY, 2.5, alignedYPath, 1)
create_nifti_image(alignedZ, 2.5, alignedZPath, 1)

CPU times: total: 1min 25s
Wall time: 1min 25s


In [51]:
%%time



CPU times: total: 0 ns
Wall time: 1min 25s


In [56]:
%%time
deformationField = np.load(os.path.join(outputDir,"deformation3d.npy"))
start = time.time()

finalX   = rg.applyDeformationField(elastixX , deformationField)
print(time.time()-start)


finalY   = rg.applyDeformationField(elastixY , deformationField)
print(time.time()-start)

finalZ   = rg.applyDeformationField(elastixZ , deformationField)

326.20450258255005
650.7775268554688
CPU times: total: 16min 16s
Wall time: 16min 17s


In [18]:
from tqdm import tqdm


In [52]:
def getTransformedData(fdata, mdata, finalX, finalY, finalZ):
    tvol = np.zeros(fdata.shape)
    count =0
    for x in tqdm(range(fdata.shape[0])):
        for y in range(fdata.shape[1]):
            for z in range(fdata.shape[2]):
                fx, fy, fz = int(finalX[x,y,z]), int(finalY[x,y,z]), int(finalZ[x,y,z])
                if fx>0 and fx<mdata.shape[0] and fy>0 and fy< mdata.shape[1] and fz>0 and fz<mdata.shape[2]:
                    tvol[x,y,z] = mdata[fx, fy, fz]
                else:
                    count+count+1
    print(count)
    return tvol

In [48]:
tvol = getTransformedData(mdata, mdata, alignedX, alignedY, alignedZ)
create_nifti_image(tvol, 2.5, "test.nii.gz", 1)

100%|████████████████████████████████████████████████████████████████████████████████| 280/280 [02:46<00:00,  1.68it/s]


In [49]:
tvol.shape

(280, 436, 562)

In [ ]:
create_nifti_image(tvol, 25, "test.nii.gz", 1)

In [53]:
eX,eY, eZ = loadNiiImages([elastixX, elastixY, elastixZ])

In [57]:
tvol=getTransformedData(fdata, mdata, finalX, finalY, finalZ)
create_nifti_image(tvol, 2.5, "test.nii.gz", 1/2)

100%|████████████████████████████████████████████████████████████████████████████████| 528/528 [03:08<00:00,  2.80it/s]


0
